In [1]:
import pandas as pd
import numpy as np
np.random.seed(143)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
treatment_status = np.random.binomial(1, 0.5, 1000)
y_under_control = np.random.normal(10, 1, 1000)
treatment_effect = np.random.normal(2, 0.1, 1000)
y_under_treatment = y_under_control + treatment_effect
y_observed = y_under_control + treatment_effect*treatment_status

In [3]:
data = pd.DataFrame({
    'treatment_status': np.random.binomial(1, 0.5, 1000),
    'y_under_control': np.random.normal(10, 1, 1000),
    'treatment_effect': np.random.normal(2, 0.1, 1000),
}).assign(
    y_under_treatment = lambda x: x.y_under_control + x.treatment_effect,
    y_observed = lambda x: x.y_under_control + x.treatment_effect*x.treatment_status
)

data.head()

,treatment_status,y_under_control,treatment_effect,y_under_treatment,y_observed
0,1,10.172693,1.758233,11.930926,11.930926
1,1,10.280985,2.062013,12.342998,12.342998
2,0,10.574830,2.017143,12.591973,10.574830
3,1,10.319585,2.043524,12.363110,12.363110
4,0,10.421973,1.827997,12.249970,10.421973


### ACE

In [4]:
data.groupby('treatment_status').y_observed.mean()

treatment_status
0    10.042473
1    12.051439
Name: y_observed, dtype: float64

In [5]:
data[['y_under_control', 'y_under_treatment']].mean()

y_under_control      10.050287
y_under_treatment    12.043651
dtype: float64

### Noncompliance

In [6]:
noncompliance_data = data.copy()
noncompliance_data['compliance'] = np.random.binomial(1, 0.5, 1000)
noncompliance_data['y_observed'] = noncompliance_data.y_under_control + noncompliance_data.treatment_effect*noncompliance_data.treatment_status*noncompliance_data.compliance

noncompliance_data.head()

,treatment_status,y_under_control,treatment_effect,y_under_treatment,y_observed,compliance
0,1,10.172693,1.758233,11.930926,10.172693,0
1,1,10.280985,2.062013,12.342998,10.280985,0
2,0,10.574830,2.017143,12.591973,10.574830,0
3,1,10.319585,2.043524,12.363110,12.363110,1
4,0,10.421973,1.827997,12.249970,10.421973,0


In [7]:
noncompliance_data.groupby('treatment_status').y_observed.mean()

treatment_status
0    10.042473
1    11.054366
Name: y_observed, dtype: float64

In [8]:
pd.crosstab(noncompliance_data.treatment_status, noncompliance_data.compliance) / 1000

compliance,0,1
treatment_status,,
0,0.275,0.262
1,0.232,0.231


### CACE

In [9]:
treated_compliers = noncompliance_data[(noncompliance_data.compliance==1) & (noncompliance_data.treatment_status==1)].shape[0]
total_treated = noncompliance_data[noncompliance_data.treatment_status==1].shape[0]
proportion_compliers = treated_compliers / total_treated

observed_outcomes = noncompliance_data.groupby('treatment_status').y_observed.mean()
cace = observed_outcomes.diff()[1] / proportion_compliers

print(f'the estimated CACE is: {cace}')

the estimated CACE is: 2.028167205694592


In [10]:
noncompliance_data.groupby(['treatment_status', 'compliance'])[['y_under_control', 'y_under_treatment']].mean()

y_under_control  y_under_treatment
treatment_status compliance                                    
0                0                 10.042668          12.029527
                 1                 10.042268          12.044713
1                0                 10.015524          12.005372
                 1                 10.103366          12.097705

In [11]:
treated_complier_mean = noncompliance_data[(noncompliance_data.treatment_status==1) & (noncompliance_data.compliance==1)].y_observed.mean()
control_mean = noncompliance_data[noncompliance_data.treatment_status==0].y_observed.mean()

treated_complier_mean - control_mean

2.0552326754025962

### Nonrandom compliance

In [12]:
treatment_status = np.random.binomial(1, 0.5, 1000)
treatment_effect = np.random.normal(2, 0.1, 1000)
y_under_control = np.random.normal(10, 1, 1000)
y_under_treatment = y_under_control + treatment_effect

q75 = np.quantile(y_under_control, q=0.75)
compliance = np.where(y_under_control >= q75, 1, 0)
y_observed = y_under_control + treatment_effect*treatment_status*compliance
treated_complier = treatment_status*compliance

nr_noncompliance_data = pd.DataFrame({
    'compliance': compliance,
    'treated_complier': treated_complier,
    'treatment_status': treatment_status,
    'treatment_effect': treatment_effect,
    'y_under_control': y_under_control,
    'y_under_treatment': y_under_treatment,
    'y_observed': y_observed
})

nr_noncompliance_data.head()

,compliance,treated_complier,treatment_status,treatment_effect,y_under_control,y_under_treatment,y_observed
0,0,0,1,1.905366,10.059335,11.964700,10.059335
1,1,0,0,2.101608,10.942760,13.044368,10.942760
2,0,0,1,2.010826,7.138338,9.149164,7.138338
3,1,1,1,2.017061,11.819005,13.836066,13.836066
4,0,0,0,2.083198,7.944678,10.027876,7.944678


In [13]:
nr_noncompliance_data.groupby(['treatment_status', 'compliance'])[['y_under_control', 'y_under_treatment']].mean()

y_under_control  y_under_treatment
treatment_status compliance                                    
0                0                  9.663208          11.668243
                 1                 11.326781          13.339515
1                0                  9.563698          11.570943
                 1                 11.330241          13.341158

In [14]:
treated_compliers = nr_noncompliance_data[nr_noncompliance_data.treated_complier==1].y_observed.mean()
control = nr_noncompliance_data[nr_noncompliance_data.treatment_status==1].y_observed.mean()

treated_compliers - control

2.798389267683973

In [15]:
ITT = nr_noncompliance_data.groupby('treatment_status').y_observed.mean().diff()[1]
proportion_compliers_in_treated = nr_noncompliance_data[nr_noncompliance_data.treated_complier==1].shape[0] / nr_noncompliance_data[nr_noncompliance_data.treatment_status==1].shape[0]
CACE = ITT / proportion_compliers_in_treated

In [16]:
CACE

1.852046400048409

### IV regression

In [17]:
from linearmodels.iv.model import IV2SLS

z = nr_noncompliance_data.treatment_status
x = nr_noncompliance_data.treated_complier
y = nr_noncompliance_data.y_observed
intercept = np.ones(len(x))

iv_fit = IV2SLS(y, intercept, x, z).fit()
iv_fit

//anaconda3/lib/python3.7/site-packages/linearmodels/iv/data.py:25: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:             y_observed   R-squared:                      0.5051
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5046
No. Observations:                1000   F-statistic:                    53.505
Date:                Thu, Dec 10 2020   P-value (F-stat)                0.0000
Time:                        13:58:11   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
exog                 10.063     0.0445     226.32     0.0000      9.9756      10.150
treated_complier     1.8520     0.2532     7.3147     0.0000      1.3558      2.3483
====================================================================================

Endogenous: treated_complier
Instruments: treatment_status
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x1a1d89e290

### Get standard errors 'by hand'

In [18]:
y_pred_control = nr_noncompliance_data[nr_noncompliance_data.treatment_status==0].y_observed.mean()
y_pred_treat_comply = y_pred_control + CACE
sst_x = ((x - x.mean())**2).sum()
r2_xz = np.corrcoef(x, z)[:1, 1]**2

y_preds = np.where(
    nr_noncompliance_data.treated_complier==0, y_pred_control, y_pred_treat_comply
)

ssr = sum((y_preds - nr_noncompliance_data.y_observed)**2)
sigma_2 = ssr / 998

In [19]:
np.sqrt(sigma_2 / (sst_x*r2_xz))

array([0.25444523])